
## To induce SP in rates

The Berekely admissions version of SP is what we will call rate based.  This requires 


2 grouping varibles & a binary
 - binary `decision` variable that will have a rate that flips (eg admission decision)
 - 1 `protected class` grouping variable that that will be how the rates are compared, eg gender (likely 2 levels, can possibly generalize)
 - 1 `explanatory` grouping variable eg has class imbalance
 
An open problem is to determine if we can define a mixed case where we use continuous variables to define the decsion variable.




In [1]:
import numpy as np
import pandas as pd
import string
import matplotlib.pylab as plt

The causal explanation for rate-based SP is that the application rates by gender varied in a way that was correlated with the department size and/or acceptance rate.  Here we create a model that will, for most samples, have SP for some departments. We only set the portion of applications to each department, the rate of each gender applying to each department and each department's acceptance rate.  

In [82]:
p_dept = [.15,.15,.1,.6]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

df = pd.DataFrame(data = data, columns=['department','gender','decision'])

p_race_dept = [[.8,.1,.1], [.7,.13,.17],[.5,.2,.3],[.85,.07,.08]]
race_list = ['W','B','H']

r = [np.random.choice(race_list, p =p_race_dept[d_i]) for d_i in d]
df['race'] = r
df.head()

,department,gender,decision,race
0,3,M,0,W
1,3,M,1,W
2,3,M,0,W
3,2,F,0,H
4,3,F,0,W


We can check that the probabilities match what we set.  First the per-department admission rate.

In [83]:
actual_admit_dept = df.groupby(['department']).mean()
expected_admit_dept = [p[0] for p in p_admit_dept]
actual_admit_dept['expected'] = expected_admit_dept
actual_admit_dept

,decision,expected
department,,
0,0.123377,0.15
1,0.229730,0.18
2,0.253165,0.25
3,0.277868,0.30


In [84]:
actual_app_dept = df.groupby(['department'])['gender'].value_counts().unstack()
actual_app_dept

gender,F,M
department,,
0,111,43
1,123,25
2,67,12
3,115,504


In [85]:
expected_app_dept_dat = [[p_d[0]*n_d, p_d[1]*n_d] for p_d,n_d in zip(p_gender_dept,[p*N for p in p_dept])]
expected_app_dept = pd.DataFrame(data = expected_app_dept_dat, columns = gender_list)
expected_app_dept

,F,M
0,105.0,45.0
1,120.0,30.0
2,85.0,15.0
3,120.0,480.0


Next we can look for SP

In [86]:
df.groupby('gender')['decision'].mean()

gender
F    0.197115
M    0.279110
Name: decision, dtype: float64

In [87]:
df.groupby(['gender','department']).mean().unstack()

decision                              
department         0         1         2         3
gender                                            
F           0.117117  0.219512  0.238806  0.226087
M           0.139535  0.280000  0.333333  0.289683

To detect SP, we compare which group is the highest in the overall to to which group is highest in each of the departments. 

In [80]:
overall_dat = df.groupby('gender')['decision'].mean()
per_dept_dat = df.groupby(['gender','department']).mean().unstack()

overall_dat.values[0]/overall_dat.values[1]
per_dept_dat.values[0]/per_dept_dat.values[1] /(overall_dat.values[0]/overall_dat.values[1])

array([7.81069891, 1.12695771, 0.68179923, 1.30806458])

In [81]:
per_dept_dat.values[0]/per_dept_dat.values[1] /(overall_dat.values[0]/overall_dat.values[1])

array([7.81069891, 1.12695771, 0.68179923, 1.30806458])

In [31]:
df.to_csv('synthetic_admission_high_sp.csv',index=False)

In [18]:
# now detect and compare which trends
overall = df.groupby('gender')['decision'].mean().idxmax()
print('overall more admitted: ', overall)

per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]
print('per dept more addmitted:', per_dept)
# it's sp for each dept that is not the same as the overall
[not(dept==overall) for dept in per_dept]

overall more admitted:  M
per dept more addmitted: ['M', 'M', 'F', 'F']


[False, False, True, True]

Now, we can run it a bunch more times and count how many SP in each trial, to see how reliable it is.

In [19]:
sp_occ = []

for i in range(20):
    p_dept = [.15,.15,.1,.6]
    p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
    gender_list = ['F','M']
    p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
    # need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
    N = 1000

    d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
    g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
    a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
    data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

    df = pd.DataFrame(data = data, columns=['department','gender','decision'])

    overall = df.groupby('gender')['decision'].mean().idxmax()
    per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]


    sp_occ.append(sum([not(dept==overall) for dept in per_dept]))
    
sp_occ

[1, 1, 1, 2, 2, 1, 2, 3, 3, 3, 2, 1, 1, 2, 2, 2, 2, 2, 2, 0]

We see from above that it generates SP most of the time but a varialbe number of times for the exact same settings to this sampling method does not reliably induce SP, it does show that, with no malintent that SP can occur.  

# intentional SP

To more reliably induce rate based SP we should sample less independently.  


In [70]:
p_dept = [.15,.2,.1,.55]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_admit_dept_gender = [{'F':.18,'M':.12},{'F':.17,'M':.1},
                       {'F':.30,'M':.27},{'F':.35,'M':.30}] 
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
p_admit =[ p_admit_dept_gender[d_i][g_i] for d_i,g_i in zip(d,g)]
p_admit
a = [np.random.choice([1,0], p = [p,1-p]) for p in p_admit]
data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

df = pd.DataFrame(data = data, columns=['department','gender','decision'])
df.head()

,department,gender,decision
0,1,M,0
1,1,M,0
2,1,F,0
3,3,M,0
4,3,M,1


In [71]:
p_admit_dept_gender[1]['M']

0.1

In [72]:
df.groupby('gender')['decision'].mean()

gender
F    0.240426
M    0.254717
Name: decision, dtype: float64

In [73]:
df.groupby(['gender','department']).mean().unstack()

decision                              
department        0         1         2         3
gender                                           
F           0.22549  0.109827  0.322917  0.404040
M           0.07500  0.075000  0.200000  0.289655

In [69]:
overall_dat = df.groupby('gender')['decision'].mean()
per_dept_dat = df.groupby(['gender','department']).mean().unstack()

overall_dat.values[0]/overall_dat.values[1]
per_dept_dat.values[0]/per_dept_dat.values[1] /(overall_dat.values[0]/overall_dat.values[1])

array([0.71104894, 2.10583975, 0.73417882, 1.61901049])

In [25]:
overall = df.groupby('gender')['decision'].mean().idxmax()

per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]


[not(dept==overall) for dept in per_dept]

[True, True, False, True]

We can try to use 3 categories and do an ordering swap detection, maybe?

In [26]:

p_race_dept = [[.8,.1,.1], [.7,.13,.17],[.5,.2,.3],[.85,.07,.08]]
race_list = ['W','B','H']

r = [np.random.choice(race_list, p =p_race_dept[d_i]) for d_i in d]
df['race'] = r

In [27]:
df.groupby('race')['decision'].mean()

race
B    0.298077
H    0.247788
W    0.254151
Name: decision, dtype: float64

In [28]:
df.groupby(['race','department']).mean().unstack()

decision                              
department         0         1         2         3
race                                              
B           0.235294  0.260870  0.227273  0.380952
H           0.076923  0.210526  0.318182  0.300000
W           0.125000  0.137931  0.306122  0.301556

In [30]:
df.head()

,department,gender,decision,race
0,1,F,0,W
1,1,M,0,W
2,3,M,1,W
3,3,M,0,H
4,3,M,1,W
